In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update

In [2]:
pwd

'C:\\Users\\C571146\\coursera\\Applied Data Science Capstone'

In [3]:
spacex_df = pd.read_csv("spacex_launch_dash.csv")

# check online with following Line:
# dash chart does not take values from options site

In [7]:

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()


# Specific for Jupyter Notebook
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

# REVIEW1: Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

# load Base Data 
data = spacex_df

# create Marker List
Marker_List = {0:0,1000:1000,2000:2000,3000:3000,4000:4000,5000:5000,6000:6000,7000:7000,8000:8000,9000:9000,9600:9600}

#Convert site into list
Site_list = spacex_df.groupby(['Launch Site']).size().reset_index(name='class count')
Site_list = list(Site_list["Launch Site"])

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Div([
                                    dcc.Dropdown(
                                        id='site-dropdown',
#                                       value='CCAFS LC-40', #this works nad gnerates graphs
#                                        options=[{'label': i, 'value': i} for i in Site_list], #this does not Work Why and the individual options do not work
                                        options=[{'label': 'ALL', 'value': 'ALL'},
                                        {'label': 'CCAFS LC-40','value':'CCAFS LC-40'},
                                        {'label': 'CCAFS SLC-40','value':'CCAFS SLC-40'},
                                        {'label': 'KSC LC-39A','value':'KSC LC-39A'},
                                        {'label': 'VAFB SLC-4E','value':'VAFB SLC-4E'}],
                                        value='ALL',
                                        placeholder="Select a Launch Site here",
                                        multi=True,
                                        searchable=True),
                                            ]),
                                    
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart',animate=True)),
                                
                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,
                                                max=10000,
                                                step=1000,
                                                marks = Marker_List,
                                                value=[min_payload, max_payload]),
                                
                                
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),

                                html.Div(id='result')
                               
                               
                               ])
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output


@app.callback(Output(component_id='result', component_property='children'),
                Input(component_id='site-dropdown', component_property='value'))


def update_result(site_dropdown):
    return (site_dropdown)

@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value'))

def pie(site_dropdown):
    data_filtered = spacex_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
    data_grouped = data_filtered[data_filtered['class']==1]
    if site_dropdown == 'ALL':
        title_pie = f"Success Launches for site {site_dropdown}"
        fig = px.pie(data_grouped, values='class count', names='Launch Site', title=title_pie)
    else:
        title_pie = f"Success Launches for site single {site_dropdown}"
        filtered_DD= data[data['Launch Site'] == site_dropdown]
        filtered_LS = filtered_DD.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig = px.pie(filtered_LS, values='class count', names='class', title=title_pie)
    return fig


# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])



def build_scatter(site,payload):
    low,high = (payload[0], payload[1])
    df = spacex_df
    filtered_Spa = spacex_df[spacex_df['Payload Mass (kg)'].between(low,high)]
    if site == 'ALL':
        title_sca = 'Payload vs. Outcome for ALL'
        fig = px.scatter(filtered_Spa,x="Payload Mass (kg)", y="class", color="Booster Version Category", title='Payload vs. Outcome for All Sites')
    else:
        title_sca = ('Payload vs. Outcome for single launch site')
        filtered_sit= filtered_Spa[filtered_Spa['Launch Site'] == site]
        fig = px.scatter(filtered_sit,x="Payload Mass (kg)", y="class", color="Booster Version Category", title=title_sca)
    return fig
 



    
if __name__ == '__main__':
    # REVIEW8: Adding dev_tools_ui=False, dev_tools_props_check=False can prevent error appearing before calling callback function
    app.run_server(mode="inline", host="localhost",port=8030, debug=False, dev_tools_ui=False, dev_tools_props_check=False)


C:\Users\C571146\.conda\envs\Python36\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

127.0.0.1 - - [30/Aug/2021 11:09:20] "GET /_shutdown_b9047f03-9164-4922-babf-9ac226db46d0 HTTP/1.1" 200 -
 * Running on http://localhost:8030/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Aug/2021 11:09:22] "GET /_alive_b9047f03-9164-4922-babf-9ac226db46d0 HTTP/1.1" 200 -


In [ ]:
def pie(site_dropdown):
    data = spacex_df
    data_filtered = spacex_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
    data_grouped = data_filtered[data_filtered['class']==1]
    if site_dropdown == 'ALL':
        title_pie = f"Success Launches for site {site_dropdown}"
        fig = px.pie(data_grouped, values='class count', names='Launch Site', title=title_pie)
        
        
    else:
        title_pie = f"Success Launches for site single {site_dropdown}"
        filtered_DD= data[data['Launch Site'] == site_dropdown]
        filtered_LS = filtered_DD.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig = px.pie(filtered_LS, values='class count', names='class', title=title_pie)
    return fig

In [ ]:
#pie('ALL')
pie('CCAFS LC-40')

In [ ]:
data.head(5)

In [ ]:
site_dropdown = 'CCAFS LC-40'
filtered_DD= data[data['Launch Site'] == site_dropdown]
filtered_LS = filtered_DD.groupby(['Launch Site','class']).size().reset_index(name='class count')
filtered_LS

In [53]:
site = 'ALL'
payload = [0,9600]
def build_scatter(site,payload):
    low,high = (payload[0], payload[1])
    df = spacex_df
    filtered_Spa = spacex_df[spacex_df['Payload Mass (kg)'].between(low,high)]
    if site == 'ALL':
        title_sca = 'Payload vs. Outcome for ALL'
        fig = px.scatter(filtered_Spa,x="Payload Mass (kg)", y="class", color="Booster Version Category", title='Payload vs. Outcome for All Sites')
    else:
        title_sca = ('Payload vs. Outcome for single launch site')
        filtered_sit= filtered_Spa[filtered_Spa['Launch Site'] == site]
        fig = px.scatter(filtered_sit,x="Payload Mass (kg)", y="class", color="Booster Version Category", title=title_sca)
    return fig

In [56]:
site_in = 'ALL'
payload_in = [0,5000]
site_sing = ('CCAFS LC-40')
build_scatter(site_sing,payload_in)

In [22]:
spacex_df.head(5)

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


In [45]:
Marker_List = {0:0,1000:1000,2000:2000,3000:3000,4000:4000,5000:5000,6000:6000,7000:7000,8000:8000,9000:9000,9600:9600}
Marker_List

{0: 0,
 1000: 1000,
 2000: 2000,
 3000: 3000,
 4000: 4000,
 5000: 5000,
 6000: 6000,
 7000: 7000,
 8000: 8000,
 9000: 9000,
 9600: 9600}

In [33]:
a_list = []
d = {}
i = 0
for l in list_payload:
    if (l not in d) and (l != '<'):
        d[l] = i
        i += 500

In [34]:
d

{0.0: 0,
 525.0: 500,
 500.0: 1000,
 677.0: 1500,
 3170.0: 2000,
 3325.0: 2500,
 2296.0: 3000,
 1316.0: 3500,
 4535.0: 4000,
 4428.0: 4500,
 2216.0: 5000,
 2395.0: 5500,
 570.0: 6000,
 4159.0: 6500,
 1898.0: 7000,
 4707.0: 7500,
 1952.0: 8000,
 2034.0: 8500,
 5271.0: 9000,
 3136.0: 9500,
 4696.0: 10000,
 3100.0: 10500,
 3600.0: 11000,
 2257.0: 11500,
 4600.0: 12000,
 553.0: 12500,
 9600.0: 13000,
 475.0: 13500,
 2150.0: 14000,
 6460.0: 14500,
 2490.0: 15000,
 5600.0: 15500,
 5300.0: 16000,
 3696.65: 16500,
 6070.0: 17000,
 2708.0: 17500,
 3669.0: 18000,
 6761.0: 18500,
 3310.0: 19000,
 4990.0: 19500,
 5200.0: 20000,
 3500.0: 20500,
 2205.0: 21000,
 4230.0: 21500,
 6092.0: 22000,
 2647.0: 22500,
 362.0: 23000,
 5384.0: 23500}

In [42]:
n = 500
ans = {i : i+500 for i in range(0,n+500)}
print(ans)

{0: 500, 1: 501, 2: 502, 3: 503, 4: 504, 5: 505, 6: 506, 7: 507, 8: 508, 9: 509, 10: 510, 11: 511, 12: 512, 13: 513, 14: 514, 15: 515, 16: 516, 17: 517, 18: 518, 19: 519, 20: 520, 21: 521, 22: 522, 23: 523, 24: 524, 25: 525, 26: 526, 27: 527, 28: 528, 29: 529, 30: 530, 31: 531, 32: 532, 33: 533, 34: 534, 35: 535, 36: 536, 37: 537, 38: 538, 39: 539, 40: 540, 41: 541, 42: 542, 43: 543, 44: 544, 45: 545, 46: 546, 47: 547, 48: 548, 49: 549, 50: 550, 51: 551, 52: 552, 53: 553, 54: 554, 55: 555, 56: 556, 57: 557, 58: 558, 59: 559, 60: 560, 61: 561, 62: 562, 63: 563, 64: 564, 65: 565, 66: 566, 67: 567, 68: 568, 69: 569, 70: 570, 71: 571, 72: 572, 73: 573, 74: 574, 75: 575, 76: 576, 77: 577, 78: 578, 79: 579, 80: 580, 81: 581, 82: 582, 83: 583, 84: 584, 85: 585, 86: 586, 87: 587, 88: 588, 89: 589, 90: 590, 91: 591, 92: 592, 93: 593, 94: 594, 95: 595, 96: 596, 97: 597, 98: 598, 99: 599, 100: 600, 101: 601, 102: 602, 103: 603, 104: 604, 105: 605, 106: 606, 107: 607, 108: 608, 109: 609, 110: 610,